In [ ]:
%pip install -q -U git+https://github.com/huggingface/transformers.git
%pip install accelerate
%pip install -i https://pypi.org/simple/ bitsandbytes
%pip install sentence-transformers

%pip install llama-index llama-index-llms-huggingface llama-index-embeddings-huggingface llama-index-readers-web

%pip install llama-index-embeddings-openai
%pip install llama-index-embeddings-adapter
%pip install llama-index-finetuning


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 954.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM


import os
import warnings

warnings.filterwarnings('ignore')

# Getting data

In [ ]:
training_file_name = "Scaling_Laws_for_Downstream_Task_Performance_of_Large_Language_Models.pdf"
validation_file_name = "Unraveling_the_Mystery_of_Scaling_Laws.pdf"

working_dir = "./"

!wget 'https://arxiv.org/pdf/2402.04177.pdf'  -O "Scaling_Laws_for_Downstream_Task_Performance_of_Large_Language_Models.pdf"
!wget 'https://arxiv.org/pdf/2403.06563.pdf' -O "Unraveling_the_Mystery_of_Scaling_Laws.pdf"

TRAIN_FILES = [os.path.join(working_dir, training_file_name)]
TRAIN_CORPUS_FPATH = "./train_corpus.json"

VAL_FILES = [os.path.join(working_dir, validation_file_name)]
VAL_CORPUS_FPATH = "./val_corpus.json"

--2024-03-26 04:59:23--  https://arxiv.org/pdf/2402.04177.pdf
Resolving arxiv.org (arxiv.org)... 151.101.3.42, 151.101.131.42, 151.101.195.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1382773 (1.3M) [application/pdf]
Saving to: ‘Scaling_Laws_for_Downstream_Task_Performance_of_Large_Language_Models.pdf’

Scaling_Laws_for_Do 100%[===================>]   1.32M  --.-KB/s    in 0.04s   

2024-03-26 04:59:23 (35.7 MB/s) - ‘Scaling_Laws_for_Downstream_Task_Performance_of_Large_Language_Models.pdf’ saved [1382773/1382773]

--2024-03-26 04:59:23--  https://arxiv.org/pdf/2403.06563.pdf
Resolving arxiv.org (arxiv.org)... 151.101.67.42, 151.101.131.42, 151.101.3.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 481993 (471K) [application/pdf]
Saving to: ‘Unraveling_the_Mystery_of_Scaling_Laws.pdf’

Unraveling_the_Myst 100%[=========

# Generating Synthetic Data

In [ ]:
import json

from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import MetadataMode

def load_corpus(files):
    reader = SimpleDirectoryReader(input_files=files)
    docs = reader.load_data()
    parser = SentenceSplitter()
    nodes = parser.get_nodes_from_documents(docs, show_progress=True)
    print(f"Parsed {len(nodes)} nodes")
    return nodes

train_nodes = load_corpus(TRAIN_FILES)
val_nodes = load_corpus(VAL_FILES)

Parsing nodes:   0%|          | 0/17 [00:00<?, ?it/s]

Parsed 23 nodes


Parsing nodes:   0%|          | 0/13 [00:00<?, ?it/s]

Parsed 17 nodes


In [ ]:
quantization_conf = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

#We recreate template (or messages) and ensure that they have the correct format, as per,
#https://github.com/run-llama/llama_index/issues/9277#issuecomment-1837545398 for zephyr-7b-beta

def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == 'system':
            prompt += f"<|system|>\n{message.content}</s>\n"
        elif message.role == 'user':
            prompt += f"<|user|>\n{message.content}</s>\n"
        elif message.role == 'assistant':
            prompt += f"<|assistant|>\n{message.content}</s>\n"

    # ensure we start with a system prompt, insert blank if needed
    if not prompt.startswith("<|system|>\n"):
        prompt = "<|system|>\n</s>\n" + prompt

    # add final assistant prompt
    prompt = prompt + "<|assistant|>\n"
    return prompt

In [ ]:
def huggingface_llm(model_name="HuggingFaceH4/zephyr-7b-beta",
                    tokenizer_name="HuggingFaceH4/zephyr-7b-beta",
                    context_window=3900,
                    max_new_tokens=256,
                    quantization_config = quantization_conf
                   ):
    llm = HuggingFaceLLM(
        model_name=model_name,
        tokenizer_name=tokenizer_name,
        query_wrapper_prompt=PromptTemplate("<|system|>\n</s>\n<|user|>\n{query_str}</s>\n<|assistant|>\n"),
        context_window=context_window,
        max_new_tokens=max_new_tokens,
        model_kwargs={"quantization_config": quantization_config},
        # tokenizer_kwargs={},
        generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
        messages_to_prompt=messages_to_prompt,
        device_map="auto",
    )

    return llm

llm = huggingface_llm()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
from llama_index.finetuning import generate_qa_embedding_pairs
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset

In [ ]:
# from llama_index.core import Settings
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Settings.llm = llm
# Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

## Preparing synthetic-training dataset

In [ ]:
train_dataset = generate_qa_embedding_pairs(train_nodes, llm)

  0%|          | 0/23 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
100%|██████████| 23/23 [04:12<00:00, 10.98s/it]


# Finetuning an embeding model

**GIST Large Embedding v0**:


In [ ]:
from llama_index.finetuning import EmbeddingAdapterFinetuneEngine
from llama_index.core.embeddings import resolve_embed_model
import torch

def embedding_model(model="local:avsolatorio/GIST-large-Embedding-v0",
                    model_output_path="model_output_test",
                    bias=True,
                    no_of_epochs=4,
                    verbose=True,
                    optimizer=torch.optim.AdamW,
                    optimizer_params={"lr": 0.01}
                   ):

    base_embed_model = resolve_embed_model(model)
    finetune_engine = EmbeddingAdapterFinetuneEngine(
        train_dataset,
        base_embed_model,
        model_output_path=model_output_path,
        bias=bias,
        epochs=no_of_epochs,
        verbose=verbose,
        optimizer_class=optimizer,
        optimizer_params=optimizer_params
    )

    return finetune_engine

finetune_engine = embedding_model()
finetune_engine.finetune()

config.json:   0%|          | 0.00/754 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

> [Epoch 1] Current loss: 2.535337448120117
> [Epoch 1] Current loss: 1.9999969005584717
> [Epoch 1] Current loss: 2.1655898094177246
> [Epoch 1] Current loss: 2.420592784881592
> [Epoch 1] Current loss: 1.4123070240020752
> [Epoch 1] Current loss: 1.2483347654342651


Iteration:   0%|          | 0/6 [00:00<?, ?it/s]

> [Epoch 2] Current loss: 2.097515821456909
> [Epoch 2] Current loss: 1.575049638748169
> [Epoch 2] Current loss: 1.556888461112976
> [Epoch 2] Current loss: 2.337671995162964
> [Epoch 2] Current loss: 1.304823398590088
> [Epoch 2] Current loss: 1.235952615737915


Iteration:   0%|          | 0/6 [00:00<?, ?it/s]

> [Epoch 3] Current loss: 1.888196349143982
> [Epoch 3] Current loss: 1.4202884435653687
> [Epoch 3] Current loss: 1.3001612424850464
> [Epoch 3] Current loss: 2.202766180038452
> [Epoch 3] Current loss: 1.2072255611419678
> [Epoch 3] Current loss: 1.219875454902649
> Finished training, saving to model_output_test


In [ ]:
finetune_engine.get_finetuned_model()

AdapterEmbeddingModel(model_name='Adapter for avsolatorio/GIST-large-Embedding-v0', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x78435d0b29b0>)

# Validation

In [ ]:
val_dataset = generate_qa_embedding_pairs(val_nodes, llm)

  0%|          | 0/17 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
100%|██████████| 17/17 [03:52<00:00, 13.65s/it]


In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core.schema import TextNode
from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
def evaluate(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
    corpus = dataset.corpus
    queries = dataset.queries
    relevant_docs = dataset.relevant_docs

    nodes = [TextNode(id_=id_, text=text) for id_, text in corpus.items()]
    index = VectorStoreIndex(
        nodes, embed_model=embed_model, show_progress=True
    )
    retriever = index.as_retriever(similarity_top_k=top_k)

    eval_results = []
    for query_id, query in tqdm(queries.items()):
        retrieved_nodes = retriever.retrieve(query)
        retrieved_ids = [node.node.node_id for node in retrieved_nodes]
        expected_id = relevant_docs[query_id][0]
        is_hit = expected_id in retrieved_ids  # assume 1 relevant doc

        eval_result = {
            "is_hit": is_hit,
            "retrieved": retrieved_ids,
            "expected": expected_id,
            "query": query_id,
        }
        eval_results.append(eval_result)
    return eval_results

In [ ]:
ada = OpenAIEmbedding()
ada_val_results = evaluate(val_dataset, ada)

Generating embeddings:   0%|          | 0/17 [00:00<?, ?it/s]

APIConnectionError: Connection error.

In [ ]:
df_ada = pd.DataFrame(ada_val_results)
hit_rate_ada = df_ada["is_hit"].mean()
hit_rate_ada

In [ ]:
GIST_model = "local:avsolatorio/GIST-large-Embedding-v0"
GIST_val_results = evaluate(val_dataset, GIST_model)
df_embed_models = pd.DataFrame(GIST_val_results)
hit_rate_bge = df_embed_models["is_hit"].mean()

Generating embeddings:   0%|          | 0/17 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 160.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 9.06 MiB is free. Process 23802 has 14.74 GiB memory in use. Of the allocated memory 14.50 GiB is allocated by PyTorch, and 101.77 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:

hit_rate_bge

In [ ]:
val_results_finetuned = evaluate(val_dataset, embed_model)
df_embed_models_finetuned = pd.DataFrame(val_results_finetuned)
hit_rate_bge_finetuned = df_embed_models["is_hit"].mean()
hit_rate_bge_finetuned

In [ ]:
results = evaluate(val_dataset, finetune_engine.get_finetuned_model())
df_embed_models = pd.DataFrame(GIST_val_results)
hit_rate = df_embed_models["is_hit"].mean()

Generating embeddings:   0%|          | 0/17 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 9.06 MiB is free. Process 23802 has 14.74 GiB memory in use. Of the allocated memory 14.59 GiB is allocated by PyTorch, and 14.96 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)